In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service # Import Service class to specify ChromeDriver location
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
import time
from bs4 import BeautifulSoup
import pandas as pd
import json
import csv

In [50]:
options = webdriver.ChromeOptions()
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-sh-usage")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)


# Open Facebook login page

In [51]:
driver.get('https://www.amazon.com/')

In [52]:
try:
    driver.get('https://www.amazon.com/ap/signin?openid.pape.max_auth_age=0&openid.return_to=https%3A%2F%2Fwww.amazon.com%2F%3Fref_%3Dnav_signin&openid.identity=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.assoc_handle=usflex&openid.mode=checkid_setup&openid.claimed_id=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0%2Fidentifier_select&openid.ns=http%3A%2F%2Fspecs.openid.net%2Fauth%2F2.0')


    email = driver.find_element(By.ID, 'ap_email')
    email.clear()
    email.send_keys('hammadaslam4101@gmail.com')
    cont = driver.find_element(By.ID, 'continue')
    cont.click()

    password = driver.find_element(By.ID, 'ap_password')
    password.clear()
    password.send_keys('mhaqwerty123')
    signin = driver.find_element(By.ID, 'signInSubmit')
    signin.click()
except:
    password = driver.find_element(By.ID, 'ap_password')
    password.clear()
    password.send_keys('mhaqwerty123')
    signin = driver.find_element(By.ID, 'signInSubmit')
    signin.click()

In [53]:
df = pd.read_csv('reviewers_links/mens_accessories_reviewers.csv')

links = df['links']

# https://www.amazon.com/gp/profile/amzn1.account.AHNMBMZ6VIRUY6MRF47PSUP4W34Q/ref=cm_cr_arp_d_gw_btm?ie=UTF8
# ids = df['reviewer_id']
    
len(links)

5557

In [54]:
with open('json/mens_clothes.json', 'r') as f:
    data = json.load(f)
cat = "Clothing, Shoes & Jewelry"
for i, link in enumerate(links):
    if i > 406 and i < len(links):
        driver.get(link)
        time.sleep(1)
        try:
            main_reviews_div = driver.find_element(By.ID, "profile-at-card-container")
            all_reviews_div = main_reviews_div.find_elements(
                By.CLASS_NAME, "your-content-card-wrapper"
            )
            # print(f'len: {len(all_reviews_div)}')
            for review in all_reviews_div:
                try:
                    div = review.find_element(By.CLASS_NAME, "your-content-card-reactions")
                    anchor = review.find_element(By.TAG_NAME, "a")
                    span = div.find_element(By.TAG_NAME, "span").text
                    if int(span) > 9:
                        a = anchor.get_attribute("href")
                        print(a)
                        # data.append(a)
                        driver.get(a)
                        time.sleep(1)
                        product_div = driver.find_element(
                            By.CSS_SELECTOR, ".a-row.a-spacing-top-mini.a-text-ellipsis"
                        )
                        product_url = product_div.find_element(
                            By.TAG_NAME, "a"
                        ).get_attribute("href")
                        driver.get(product_url)
                        time.sleep(1)
                        category_div = driver.find_element(
                            By.ID, "wayfinding-breadcrumbs_feature_div"
                        )
                        category = category_div.find_element(By.TAG_NAME, "a").text
                        #######################
                        # print('1')
                        #######################
                        if cat not in category:
                            driver.back()
                        else:
                            #######################
                            print('entered else')
                            #######################
                            product_title = driver.find_element(By.ID, "productTitle").text
                            #######################
                            print(f'title received: {product_title}')
                            #######################
                            average_rating = float(
                                driver.find_element(By.ID, "acrPopover")
                                .get_attribute("title")
                                .split(" ")[0]
                            )
                            #######################
                            print(f'average rating received: {average_rating}')
                            #######################
                            total_rating = int(
                                driver.find_element(By.ID, "acrCustomerReviewText")
                                .text.split(" ")[0]
                                .replace(",", "")
                            )
                            #######################
                            print(f'total rating received: {total_rating}')
                            #######################
                            categories_div = driver.find_elements(
                                By.CSS_SELECTOR,
                                ".a-unordered-list.a-horizontal.a-size-small > li:not([class])",
                            )
                            #######################
                            print(f'categories div received: {len(categories_div)}')
                            #######################
                            product_category = []
                            for el in categories_div:
                                anc = el.find_element(By.TAG_NAME, "a").text
                                #######################
                                print(f'anchor found in categories div: {anc}')
                                #######################
                                product_category.append(anc)
                            product_category = ", ".join(product_category)
                            #######################
                            print(f'product_category: {product_category}')
                            #######################
                            details_div = driver.find_element(
                                By.CSS_SELECTOR,
                                ".a-expander-content.a-expander-partial-collapse-content",
                            )
                            #######################
                            print(f'details div received: {details_div.get_attribute('data-expanded')}')
                            #######################
                            feature_divs = details_div.find_elements(By.CSS_SELECTOR, '.a-fixed-left-grid.product-facts-detail')
                            #######################
                            print(f'feature div received: {len(feature_divs)}')
                            #######################
                            features = {}
                            for feat in feature_divs:
                                feat_head = feat.find_element(By.CSS_SELECTOR, '.a-fixed-left-grid-col.a-col-left')
                                feat_name = feat_head.find_element(By.CLASS_NAME, 'a-color-base').text
                                #######################
                                # print(f'feat head received: {feat_name}')
                                #######################
                                feat_tail = feat.find_element(By.CSS_SELECTOR, '.a-fixed-left-grid-col.a-col-right')
                                feat_value = feat_tail.find_element(By.CLASS_NAME, 'a-color-base').text
                                #######################
                                # print(f'feat value received: {feat_value}')
                                #######################
                                features[feat_name] = feat_value
                            #######################
                            print(f'features received: {features}')
                            #######################
                            # desc_more = details_div.find_element(By.ID, 'productFactsToggleButton')
                            # desc_a = desc_more.find_element(By.TAG_NAME, 'a')
                            # #######################
                            # print(f'more description found: {desc_a.get_attribute('href')}')
                            # #######################
                            # desc_a.click()
                            # #######################
                            # print('more description clicked')
                            # #######################
                            desc_div = details_div.find_elements(By.CSS_SELECTOR, '.a-unordered-list.a-vertical.a-spacing-small')
                            #######################
                            print(f'description div received: {len(desc_div)}')
                            #######################
                            description = []
                            for el in desc_div:
                                li = el.find_element(By.TAG_NAME, 'li')
                                #######################
                                print('got list item')
                                #######################
                                desc = li.find_element(By.TAG_NAME, 'span').text
                                #######################
                                print(f'got desctiption: {desc}')
                                #######################
                                description.append(desc)
                            description = (", ").join(description)
                            product_asin = driver.find_element(By.XPATH, "//div[@id='detailBullets_feature_div']//span[contains(text(), 'ASIN')]/following-sibling::span").text
                            #######################
                            print(f'got product asin: {product_asin}')
                            #######################
                            
                            driver.back()
                            #######################
                            print('get back')
                            #######################
                            time.sleep(1)
                            user_name = driver.find_element(By.CLASS_NAME, 'a-profile-name').text
                            #######################
                            print(f'user name received: {user_name}')
                            #######################
                            review_title = driver.find_element(By.CSS_SELECTOR, "a.a-size-base.a-link-normal.review-title.a-color-base.review-title-content.a-text-bold > span:not([class])").text
                            #######################
                            print(f'review title received: {review_title}')
                            #######################
                            verified_purchase = True if driver.find_element(By.CSS_SELECTOR, '.a-size-mini.a-color-state.a-text-bold').text == 'Verified Purchase' else False
                            #######################
                            print(f'verified purchase received: {verified_purchase}')
                            #######################
                            review_rating_anchor = driver.find_element(By.CSS_SELECTOR, "a.a-size-base.a-link-normal.review-title.a-color-base.review-title-content.a-text-bold")
                            review_rating = review_rating_anchor.find_element(By.TAG_NAME, "i")
                            review_rating = review_rating.get_attribute('class')
                            # a-icon a-icon-star a-star-5 review-rating
                            review_rating = review_rating.split(' ')[2]
                            review_rating = review_rating.split('-')[2]
                            # review_rating = review_icon.find_element(By.CSS_SELECTOR, '.a-icon-alt').text
                            #######################
                            print(f'review rating received: {review_rating}')
                            #######################
                            text_span = driver.find_element(By.CSS_SELECTOR, '.a-size-base.review-text.review-text-content')
                            #######################
                            print('got review body div')
                            #######################
                            review_text = text_span.find_element(By.TAG_NAME, 'span').text.replace('\n<br>\n<br>\n', ' ')
                            #######################
                            print(f'review body received: {review_text}')
                            #######################
                            helpful_votes = driver.find_element(By.CSS_SELECTOR, '.a-size-base.a-color-tertiary.cr-vote-text').text.split(' ')[0]
                            helpful_votes = 1 if helpful_votes == 'One' else int(helpful_votes)
                            #######################
                            print(f'got helpful votes: {helpful_votes}')
                            #######################
                            
                            row = {
                                'user_name': user_name,
                                'user_id': link.split('/ref')[0].split('account.')[1],
                                'product_title': product_title,
                                'average_rating': average_rating,
                                'total_rating': total_rating,
                                'product_category': product_category,
                                'features': features,
                                'description': description,
                                'product_asin': product_asin,
                                'review_title': review_title,
                                'verified_purchase': verified_purchase,
                                'review_rating': float(review_rating),
                                'text': review_text,
                                'helpful_votes': helpful_votes
                            }
                            data.append(row)
                            time.sleep(1)
                            print(row)
                            with open('json/mens_clothes.json', 'w') as json_file:
                                json.dump(data, json_file, indent=4)
                            with open('json/mens_clothes.csv', 'w', newline='') as csv_file:
                                # Initialize the writer with the field names
                                fieldnames = ['Name of Customer', 'Customer ID', 'Product Title', 'Product Average Rating', 'Total No. of Rating', 'Product Categories', 'Product Features', 'Product Description', 'Product ASIN', 'Review Title', 'Is Verified', 'Review Rating', 'Review Text', 'Helpful Votes']
                                writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

                                # Write the header only once (this clears the file each time)
                                writer.writeheader()
                                
                                # Write all rows
                                writer.writerows(data)
                        driver.back()
                except:
                    continue
        except:
            continue
        
        print(f"{i} : reviewers done")

336 : reviewers done
337 : reviewers done
338 : reviewers done
339 : reviewers done
340 : reviewers done
341 : reviewers done
342 : reviewers done
343 : reviewers done
344 : reviewers done
345 : reviewers done
346 : reviewers done
347 : reviewers done
348 : reviewers done
349 : reviewers done
350 : reviewers done
351 : reviewers done
352 : reviewers done
353 : reviewers done
354 : reviewers done
355 : reviewers done
356 : reviewers done
357 : reviewers done
358 : reviewers done
359 : reviewers done
360 : reviewers done
361 : reviewers done
362 : reviewers done
363 : reviewers done
364 : reviewers done
365 : reviewers done
366 : reviewers done
367 : reviewers done
368 : reviewers done
369 : reviewers done
370 : reviewers done
371 : reviewers done
372 : reviewers done
373 : reviewers done
374 : reviewers done
375 : reviewers done
376 : reviewers done
https://www.amazon.com/gp/customer-reviews/R1TB2YD4Y909FN?ref=pf_vv_at_pdctrvw_srp
377 : reviewers done
378 : reviewers done
https://www.am

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0.6533.100)
Stacktrace:
	GetHandleVerifier [0x006E8923+23283]
	(No symbol) [0x006AE934]
	(No symbol) [0x005E0733]
	(No symbol) [0x005BD2E3]
	(No symbol) [0x0064A64F]
	(No symbol) [0x0065C686]
	(No symbol) [0x006441B6]
	(No symbol) [0x00618017]
	(No symbol) [0x0061890D]
	GetHandleVerifier [0x007DA5E3+1013683]
	GetHandleVerifier [0x007E3E3C+1052684]
	GetHandleVerifier [0x007DD4A4+1025652]
	GetHandleVerifier [0x0070EA2B+179195]
	(No symbol) [0x006B6833]
	(No symbol) [0x006B3198]
	(No symbol) [0x006B3337]
	(No symbol) [0x006AB4BE]
	BaseThreadInitThunk [0x7599FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x775880CE+286]
	RtlGetAppContainerNamedObjectPath [0x7758809E+238]


In [36]:
len(data)

38